<a href="https://colab.research.google.com/github/jmmarinr/ComputationalMethods/blob/master/Algebra_Lineal/Actividad_08_Algrebra_Matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la

# Actividad 08: Algebra Lineal y Matrices

---
### Profesor: Juan Marcos Marín
### Nombre: Jesus Armando Cañas Gamboa
*Métodos computacionales*

---

#1
Escriba tres matrices aleatorias $A$, $B$ y $C$ de $3\times 3$, y demuestre las siguientes relaciones

- $ \mathbf{A}\mathbf{B} \neq \mathbf{B}\mathbf{A} $, en general.
- $ (\mathbf{A}\mathbf{B})\mathbf{C} = \mathbf{A}(\mathbf{B}\mathbf{C}) $.
- $ \mathbf{A}(\mathbf{B} + \mathbf{C}) = \mathbf{A}\mathbf{B} + \mathbf{A}\mathbf{C} $.
- $ (\mathbf{A} + \mathbf{B})\mathbf{C} = \mathbf{A}\mathbf{C} + \mathbf{B}\mathbf{C} $.
- $ (\mathbf{A}\mathbf{B})^\top = \mathbf{B}^\top \mathbf{A}^\top $.
- $ \det(\mathbf{A}\mathbf{B}) = \det(\mathbf{A}) \det(\mathbf{B}) $.
- $ (\mathbf{A}^\top)^\top = \mathbf{A} $.
- $ (c\mathbf{A})^\top = c\mathbf{A}^\top $.
- $ (\mathbf{A} + \mathbf{B})^\top = \mathbf{A}^\top + \mathbf{B}^\top $.



In [90]:

A = np.random.rand(3, 3)
B = np.random.rand(3, 3)
C = np.random.rand(3, 3)
c = np.random.rand()

print(np.allclose((A@B).T, B.T @ A.T))
print(np.allclose(np.linalg.det(A@B), np.linalg.det(A) * np.linalg.det(B)))
print(np.array_equal((A.T).T, A))
print(np.array_equal((c*A).T, c*A.T))
print(np.array_equal((A+B).T, A.T + B.T))


True
True
True
True
True


In [20]:
np.array_equal(A,A)

True

In [12]:
 A@(B@C)

array([[0.70214591, 0.41980988, 1.08968598],
       [1.13382826, 0.57022048, 1.65216725],
       [1.10696605, 0.49266532, 1.52881041]])

#2

El **Teorema de Laplace** es un método para calcular el determinante de una matriz cuadrada, particularmente útil para matrices de orden mayor a 2. Este teorema se basa en la expansión del determinante por los elementos de una fila o una columna cualquiera.



$$
\det(A) = \sum_{j=1}^n (-1)^{1+j} a_{1j} M_{1j}
$$

donde:
- $a_{1j}$ es el elemento de la primera fila y columna $j$.
- $M_{1j}$ es el menor asociado al elemento $a_{1j}$, es decir, el determinante de la submatriz de $3 \times 3$ que se obtiene al eliminar la fila 1 y la columna $j$.
- $(-1)^{1+j}$ es el signo correspondiente al cofactor del elemento $a_{1j}$.

Podemos realizar una función recursiva para el cálculo del determinante, sabiendo que el valor del determinante de una matriz de orden uno es el único elemento de esa matriz, y el de una matriz de orden superior a uno es la suma de cada uno de los elementos de una fila o columna por los Adjuntos a ese elemento, como en la función recursiva se emplea la misma función definida el cálculo lo haremos por Menor complementario, un ejemplo desarrollado por la primera fila sería:

$$
   \det (A_{j,j}) =
   \left \{
   \begin{array}{llcl}
      si & j = 1 & \to & a_{1,1} \\
                                 \\
      si & j > 1 & \to & \displaystyle \sum_{k=1}^j \; (-1)^{(1+k)} \cdot a_{1,k} \cdot \det( \alpha_{1,k})
   \end{array}
   \right .
$$

Realice una función que encuentre el determinante de una matriz usando la recursividad aqui planteada, explique explicitamente su código

In [11]:
def Teorema_de_Laplace(A):

    '''
    Esta funcion calcula el determinante de una
    matriz usando el teorema de laplace

    Entradas:
    A: Matriz a la que se le calculara el determinante

    Salidas:
    suma: valor del determinante

    '''
    if A.shape[0] != A.shape[1]: #miramos si la matriz es cuadrada
        raise ValueError("La matriz no es cuadrada") 

    n = A.shape[0] # si es cuadrada calculamos el tamaño
    if n == 1: # si el tamaño de la matriz es 1x1 retornamos al valor del elemento
        return A[0, 0]

    suma = 0
    for k in range(n):
        # Construimos los índices sin la fila 0 y sin la columna k
        A1 = A[1:n, 0:k] 
        A2 = A[1:n, k+1:n+1]
        submatriz = np.concatenate((A1, A2), axis = 1)  
        suma += (-1)**k * A[0, k] * Teorema_de_Laplace(submatriz) #Calculamos el teorema de laplace usando recursividad

    return suma
        

In [12]:
print(Teorema_de_Laplace(A))
print(np.linalg.det(A))

-0.25809870446108435
-0.25809870446108435


#3 Método de Gauss - Seidel

Sea \$A\in\mathbb{R}^{n\times n}\$ no singular y sea \$b\in\mathbb{R}^n\$.
Descomponga \$A\$ como

$$
A \;=\; D \;+\; L \;+\; U,
$$

donde

* \$D\$ es la matriz diagonal de \$A\$,
* \$L\$ es la parte estrictamente triangular inferior,
* \$U\$ es la parte estrictamente triangular superior.

El algoritmo de Gauss - Seidel reorganiza el sistema \$Ax=b\$ como

$$
x \;=\; (D+L)^{-1}\bigl(b \;-\; Ux\bigr),
$$

y genera la sucesión

$$
x_i^{(k+1)}
= \frac{1}{a_{ii}}
\Bigl(b_i - \sum_{j<i} a_{ij}\,x_j^{(k+1)} - \sum_{j>i} a_{ij}\,x_j^{(k)}\Bigr),
\qquad i=1,\dots,n.
$$

Implemente una función `gauss_seidel(A, b, tol=1e-7, max_iter=100)` que:
   * Realice las iteraciones hasta que
     $\lVert x^{(k+1)}-x^{(k)}\rVert_\infty<\text{tol}$
     o se alcance `max_iter`;
   * devuelva el vector solución aproximado \$x\$, el número de iteraciones realizadas y la norma del último residuo.

Incluya una documentación clara.

Luego,

   * Genere una matriz aleatoria \$5\times5\$ (por ejemplo, con `np.random.rand`) y un vector \$b\$ aleatorio.
   * Resuelva \$Ax=b\$ con su función; calcule el error relativo frente a `numpy.linalg.solve`.
   * Estime igualmente el error respecto a la solución obtenida mediante \$x=A^{-1}b\$ (usando `numpy.linalg.inv`).
   * Presente las normas de los residuos y los errores relativos.

In [77]:
def gauss_seidel(A, b, tol=1e-7, max_iter=300):
    ''' Esta funcion resuelve un sistema de la forma Ax = b
    usando el metodo de Gauss seidel
    entradas
       A: matriz(array)
       b : vector(array)
       tol: tolerancia (float)
       max_iter = numero maximo de iteraciones(float)
    salida
        x : vector(array)
        j : numero de iteraciones hasta converger
    '''
    n = len(b)
    x = np.zeros_like(b, dtype=float)
    x_new = np.zeros_like(x)

    for k in range(max_iter):
        for i in range(n):
            suma_1 = 0.0
            suma_2 = 0.0
            
            for j in range(n):
                if j < i:
                    suma_1 += A[i, j] * x_new[j]  # usa valores nuevos
                if j > i:
                    suma_2 += A[i, j] * x[j]      # usa valores viejos
            x_new[i] = (b[i] - suma_1 - suma_2) / A[i, i]

        # Verificamos convergencia con norma infinito
        error = np.linalg.norm(x_new - x, ord=np.inf)
        if error < tol:
            return x_new, k
        else:
            x[:] = x_new  # actualizamos x para la siguiente iteración
    
    raise ValueError('maximo de iteraciones alcanzado')

    


In [89]:


np.random.seed(1)
n = 5
A = np.random.rand(n, n)
for i in range(n):
    A[i, i] = np.sum(np.abs(A[i, :])) + 1.0
b = np.random.rand(n)

x_gs, it = gauss_seidel(A, b)
x_np = np.linalg.solve(A, b)
x_inv = np.linalg.inv(A) @ b

r_gs = b - A @ x_gs
r_np = b - A @ x_np
r_inv = b - A @ x_inv

residuals = {
    'GS_norm2': np.linalg.norm(r_gs, 2),
    'GS_norm_inf': np.linalg.norm(r_gs, np.inf),
    'NP_norm2': np.linalg.norm(r_np, 2),
    'NP_norm_inf': np.linalg.norm(r_np, np.inf),
    'INV_norm2': np.linalg.norm(r_inv, 2),
    'INV_norm_inf': np.linalg.norm(r_inv, np.inf),
}

rel_errors_vs_np = {
    'GS_rel_error_2': np.linalg.norm(x_gs - x_np, 2) / np.linalg.norm(x_np, 2),
    'GS_rel_error_inf': np.linalg.norm(x_gs - x_np, np.inf) / np.linalg.norm(x_np, np.inf),
    'INV_rel_error_2': np.linalg.norm(x_inv - x_np, 2) / np.linalg.norm(x_np, 2),
    'INV_rel_error_inf': np.linalg.norm(x_inv - x_np, np.inf) / np.linalg.norm(x_np, np.inf),
}

rel_between_gs_inv = {
    'GS_vs_INV_2': np.linalg.norm(x_gs - x_inv, 2) / np.linalg.norm(x_inv, 2),
    'GS_vs_INV_inf': np.linalg.norm(x_gs - x_inv, np.inf) / np.linalg.norm(x_inv, np.inf),
}

print("A =\n", A)
print("\nb =", b)
print("\nsol_gauss_seidel =", x_gs)
print("iteraciones =", it)
print("\nsol_numpy =", x_np)
print("\nsol_inv =", x_inv)
print("\nNormas de los residuos =", residuals)
print("\nErrores relativos vs numpy =", rel_errors_vs_np)
print("\nErrores relativos GS vs INV =", rel_between_gs_inv)


A =
 [[2.58654934e+00 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01]
 [9.23385948e-02 2.55974374e+00 3.45560727e-01 3.96767474e-01
  5.38816734e-01]
 [4.19194514e-01 6.85219500e-01 3.21437129e+00 8.78117436e-01
  2.73875932e-02]
 [6.70467510e-01 4.17304802e-01 5.58689828e-01 2.98495057e+00
  1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01
  4.65114209e+00]]

b = [0.89460666 0.08504421 0.03905478 0.16983042 0.8781425 ]

sol_gauss_seidel = [ 3.41025089e-01 -2.97457169e-04 -2.69658082e-02 -2.36069594e-02
  1.35483261e-01]
iteraciones = 9

sol_numpy = [ 3.41025082e-01 -2.97458619e-04 -2.69658069e-02 -2.36069575e-02
  1.35483263e-01]

sol_inv = [ 3.41025082e-01 -2.97458619e-04 -2.69658069e-02 -2.36069575e-02
  1.35483263e-01]

Normas de los residuos = {'GS_norm2': 1.885562130622038e-08, 'GS_norm_inf': 1.854012643409675e-08, 'NP_norm2': 1.6782945342419286e-16, 'NP_norm_inf': 1.1102230246251565e-16, 'INV_norm2': 7.110250420355456e-17, 'INV_norm_in

#4 Método de potencias para el valor propio dominante

Sea \$A\in\mathbb{R}^{n\times n}\$ diagonalizable con valor propio dominante \$\lambda\_{\max}\$ (en magnitud) y vector propio asociado \$v\_{\max}\$.

El método de potencias genera, a partir de un vector inicial \$q^{(0)}\neq 0\$, la sucesión

$$
q^{(k+1)} \;=\; \frac{A\,q^{(k)}}{\lVert A\,q^{(k)}\rVert_2},
\qquad
\lambda^{(k+1)} \;=\; (q^{(k+1)})^{\!\top} A\, q^{(k+1)},
$$

que converge a \$v\_{\max}/\lVert v\_{\max}\rVert\_2\$ y a \$\lambda\_{\max}\$ respectivamente, bajo hipótesis estándar.

Implemente `power_method(A, tol=1e-7, max_iter=1000)` que:

   * Acepte matrices reales cuadradas,
   * Devuelva \$\lambda\_{\max}\$, el vector propio normalizado \$v\_{\max}\$, el número de iteraciones y la última variación relativa de \$\lambda\$,
   * detenga la iteración cuando
     $\bigl|\lambda^{(k+1)}-\lambda^{(k)}\bigr|<\text{tol}\,|\lambda^{(k+1)}|$
     o se alcance `max_iter`.

Luego,
   * Genere una matriz simétrica aleatoria \$6\times6\$ (por ejemplo, \$A = (M+M^\top)/2\$ con \$M\$ aleatoria).
   * Aplique su `power_method` y compare \$\lambda\_{\max}\$ y \$v\_{\max}\$ con los resultados de `numpy.linalg.eig`.

In [23]:
def power_method(A, tol=1e-7, max_iter=1000):

    if A.dtype != float:
        raise ValueError('La matriz no es real')
    if A.shape[0] !=A.shape[1]:
        raise ValueError(' La matriz no es cuadrada')
    n = len(A)

    q = np.ones(n)
    q_new = np.zeros_like(q)
    lamda = 0
    for i in range(max_iter):
        q_new = A@q/np.linalg.norm(A@q)
        lamda_new = q_new.T @ (A @ q_new)

        if abs(lamda_new - lamda) < tol * lamda_new:
            return q_new, lamda_new, i, lamda
        lamda = lamda_new
        q = q_new

In [28]:
M = np.random.rand(6,6)
A = (M + M.T)/2

v, l, itera, ulland = power_method(A)
print('usando power method')
print("Autovalor dominante:", l)
print("Autovector dominante:", v)

l, v = np.linalg.eig(A)
#Encontramos el indice del autovalor propio maximo
idx = np.argmax(l)

#Encontramos el valor y vector propio
lmax = l[idx]
vmax = v[:, idx]

print('usando linalg.eig')
print("Autovalor dominante:", lmax)
print("Autovector dominante:", abs(vmax))
    

usando power method
Autovalor dominante: 2.6694855187791577
Autovector dominante: [0.4934119  0.36928283 0.45227796 0.38323565 0.34909252 0.38325498]
usando linalg.eig
Autovalor dominante: 2.6694855212382875
Autovector dominante: [0.49341409 0.3692752  0.45228132 0.38323599 0.34907396 0.38327212]


#5

Verifique que cualquier matriz hermitiana de 2 × 2 $ L $ puede escribirse como una suma de cuatro términos:

$$ L = a\sigma_x + b\sigma_y + c\sigma_z + dI $$

donde $ a $, $ b $, $ c $ y $ d $ son números reales.

Las cuatro matrices de Pauli son:

$$ \sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad \sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad \sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}, \quad I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} $$




In [35]:
# Bases: Pauli + identidad
sigma_x = np.array([[0, 1],
                    [1, 0]], dtype=complex)

sigma_y = np.array([[0, -1j],
                    [1j, 0]], dtype=complex)

sigma_z = np.array([[1, 0],
                    [0, -1]], dtype=complex)

I = np.eye(2, dtype=complex)
#Planteamos un sistema de ecuaciones Ax = b
#donde x son los coeficientes de pauli

# Ejemplo con una matriz hermitica 2x2
H = np.array([[2 + 1j, 0.5 - 0.2j],
              [1.2 + 0.3j, -0.7 - 0.5j]], dtype=complex)

#Matriz A asociada al sistema
A = np.array([[0, 0, 1, 1],
              [1, -1j, 0, 0],
              [1, 1j, 0, 0],
              [0, 0, -1, 1]], dtype = complex)

B = np.array([H[0,0], H[0,1], H[1,0], H[1,1]])

a, b, c, d = np.linalg.solve(A, B)


print("Coeficientes (a,b,c,d):", a, b, c, d)

# Reconstruir y comprobar
H_new = a*sigma_x + b*sigma_y + c*sigma_z + d*I
print("\n¿Coincide H con la reconstrucción?:", np.allclose(H, H_new))
print("\nH original:\n", H)
print("\nH reconstruida:\n", H_new)


Coeficientes (a,b,c,d): (0.85+0.04999999999999999j) (0.25-0.35j) (1.35+0.75j) (0.65+0.25j)

¿Coincide H con la reconstrucción?: True

H original:
 [[ 2. +1.j   0.5-0.2j]
 [ 1.2+0.3j -0.7-0.5j]]

H reconstruida:
 [[ 2. +1.j   0.5-0.2j]
 [ 1.2+0.3j -0.7-0.5j]]


# 6

Haga un breve resumen en Markdown de las funciones y métodos más relevantes para algebra lineal usando Python. Emplee ejemplos.

# Funciones y métodos más relevantes para algebra lineal usando Python.

| Método | Descripción | Ejemplo de Uso | Caso de Aplicación Típico |
|--------|-------------|----------------|---------------------------|
| `np.linalg.solve` | Resuelve un sitema de ecuaciones | `np.linag.solve(A, b)` | Resolver sistema Ax = b |
| `np.linalg.eig` | Encuentra los valores y vectores propios de una matriz cuadrada | `np.linalg.eig(A)` | Solucion de ecuaciones diferenciales lineales de primer orden |
| `.T` | Transpone una matriz | `A.T` | Intercambiar filas y columnas |
| `np.random.rand()` | sirve para obtener una matriz aleatoria de cierto tamaño | `np.random.rand(5,5)` | Matriz aleatoria de tamaño 5x5 |
| `Power_metho` | Calcula los valores y vectores propios de una matriz | `Power_method(A)` | lo mismo que linalg.eig |
| `.shape` | sirve para obtener las dimensiones de una matriz | `A.shape` | para hacer metodos iterativos con matrices |
| `len` | sirve para obtener la cantidad de elementos de cada fila de una matriz | `len(A)` | Para hecer metodos iterativos |
| `np.zeros_like` | Sirve para crear matrices vacias con la dimension de una matriz A | `np.zeros_like(A)` | Para metodos iterativos |
| `np.diag()` | Extrae o crea diagonal | `np.diag(A)` | Manejo de matrices diagonales |
| `np.linalg.det()` | Calcula el determinante de una matriz | `np.linalg.det(A)` | Resolución de sistemas lineales |
| `@` | Multiplicación matricial | `A@B` | Multiplicación de matrices |
| `np.inner()` | Producto interno | `np.inner(a, b)` | Proyecciones y ángulos entre vectores |
| `np.norm` | sirve para calcularle la norma a un vector | `np.norm(v)` |Para emcontrar los errores de metodos, para encotrar bases ortonormales |
| `np.allclose` | comparacion de matrices | `np.allclose(A,B) ` | para comparar la eficiencia de los metodos |